In [443]:
import pandas as pd

from snapgene_reader import snapgene_file_to_dict
from Bio import Seq
import os
from collections import Counter
import re

grna = pd.read_excel("C:/Users/최민재/Useful_Codes/Cas9 crRNA Finder for Clean KO/alkB_target_3.xlsx")

file_path = "C:/Users/최민재/Useful_Codes/Cas9 crRNA Finder for Clean KO/E. coli str. K-12 substr. W3110.dna"
dictionary = snapgene_file_to_dict(file_path)
genome_seq = dictionary['seq']


target_df = grna.loc[:, ['target']]

seed_df = pd.DataFrame(index = range(0,0),columns = ["seed"])

PAM_df = pd.DataFrame(index = range(0,0),columns = ["PAM"])


i=0

num = len(target_df)

while i<num:
    seed = target_df.iloc [i]['target'][10:]
    seed_df.loc[i,'seed'] = seed
    i=i+1



seq = Seq.Seq (genome_seq)
length = len(genome_seq)
genome_rev = str(seq.reverse_complement())


PAM_1 = 'GG'
PAM_2 = 'AG'

for p in re.finditer('(?=.[AG]G)', genome_seq):
    q = p.start()
    PAM_target = genome_seq[q-10:q]
    if (seed_df['seed']==PAM_target).any() == True:
        PAM_df = pd.concat([PAM_df, pd.DataFrame({'PAM' : [PAM_target]})], ignore_index=True)

for p in re.finditer('(?=.[AG]G)', genome_rev):
    q = p.start()
    PAM_target = genome_rev[q-10:q]
    if (seed_df['seed']==PAM_target).any() == True:
        PAM_df = pd.concat([PAM_df, pd.DataFrame({'PAM' : [PAM_target]})], ignore_index=True)

print (PAM_df)

PAM_df['PAM'].value_counts()

df = PAM_df['PAM'].value_counts().rename_axis('seed').reset_index(name='off_target')

j = 0
k = 0

print (df)

df_2 = df.copy()

df_2.insert(0, 'crRNA', '')

while j < len(df):
    seed = df.iloc [j,0]
    while k < len(df):
        cr = target_df.iloc [k,0]
        if seed == cr[10:]:
            df_2.iloc[j,0] = cr
        k = k+1
    k = 0
    j = j+1

print(df_2)

df_2.to_excel("C:/Users/최민재/Useful_Codes/Cas9 crRNA Finder for Clean KO/alkB_crRNA_final.xlsx")

           PAM
0   GAGCCACTGG
1   TATCCCCCGG
2   TATTTGTTTG
3   TGCGGATCAA
4   CCTTGCCGAT
5   GTATATCCCC
6   TATCCCCCGG
7   GGTGACCATC
8   TGCGGATCAA
9   TATCCCCCGG
10  GGTGACCATC
11  CCTTGCCGAT
12  ACATTCCGTC
13  GGTGACCATC
14  TATCCCCCGG
15  CAAGAGCCAC
16  GAGCCACTGG
17  TATACCATGT
18  GACGACCCAT
19  ACATTCCGTC
20  TATTTGTTTG
21  TGCGGATCAA
22  TGCGGATCAA
23  GACGACCCAT
24  TGCGGATCAA
          seed  off_target
0   TGCGGATCAA           5
1   TATCCCCCGG           4
2   GGTGACCATC           3
3   GAGCCACTGG           2
4   TATTTGTTTG           2
5   CCTTGCCGAT           2
6   ACATTCCGTC           2
7   GACGACCCAT           2
8   GTATATCCCC           1
9   CAAGAGCCAC           1
10  TATACCATGT           1
                   crRNA        seed  off_target
0   TTTATTTGTTTGCGGATCAA  TGCGGATCAA           5
1   CGACATGGTATATCCCCCGG  TATCCCCCGG           4
2   ATCCCCCGGGGGTGACCATC  GGTGACCATC           3
3   ACCGTGGCAAGAGCCACTGG  GAGCCACTGG           2
4   GGCCACGGTTTATTTGTTTG  TATTTGTTTG     